Use SQLAlchemy to model your table schemas and create a sqlite database for your tables. You will need one table for measurements and one for stations.

Create a Jupyter Notebook called database_engineering.ipynb and use this to complete all of your Database Engineering work.

Use Pandas to read your cleaned measurements and stations CSV data.

Use the engine and connection string to create a database called hawaii.sqlite.


In [1]:
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
measurements= pd.read_csv("clean_Resources_hawaii_measurements.csv")
measurements.head()

,station,date,prcp,tobs
0,USC00519397,2010-01-01,0.08,65
1,USC00519397,2010-01-02,0.00,63
2,USC00519397,2010-01-03,0.00,74
3,USC00519397,2010-01-04,0.00,76
4,USC00519397,2010-01-07,0.06,70


In [3]:
station= pd.read_csv("clean_Resources_hawaii_stations.csv")
station

,station,name,latitude,longitude,elevation
0,USC00519397,"WAIKIKI 717.2, HI US",21.27160,-157.81680,3.0
1,USC00513117,"KANEOHE 838.1, HI US",21.42340,-157.80150,14.6
2,USC00514830,"KUALOA RANCH HEADQUARTERS 886.9, HI US",21.52130,-157.83740,7.0
3,USC00517948,"PEARL CITY, HI US",21.39340,-157.97510,11.9
4,USC00518838,"UPPER WAHIAWA 874.3, HI US",21.49920,-158.01110,306.6
5,USC00519523,"WAIMANALO EXPERIMENTAL FARM, HI US",21.33556,-157.71139,19.5
6,USC00519281,"WAIHEE 837.5, HI US",21.45167,-157.84889,32.9
7,USC00511918,"HONOLULU OBSERVATORY 702.2, HI US",21.31520,-157.99920,0.9
8,USC00516128,"MANOA LYON ARBO 785.2, HI US",21.33310,-157.80250,152.4


In [4]:
import sqlalchemy
from sqlalchemy import create_engine, MetaData
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, Text, Float
Base = declarative_base()

In [5]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [6]:
conn = engine.connect()

In [7]:
class Measurements(Base):
    __tablename__ = "Measurements"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    date = Column(String) # for late using the date column, we need to use pandas to convert it back to datetime format for analysis 
    prcp = Column(Float)
    tobs = Column(Float)
    
    def __repr__(self):
        return f"id={self.id}, station = {self.station}, date ={self.date}, prcp = {self.prcp}, tobs = {self.tobs}"

In [8]:
class Station(Base):
    __tablename__ = "Station"
    id = Column(Integer, primary_key=True)
    station = Column(String)
    name = Column(String)
    latitude = Column(Float)
    longitude = Column(Float)
    elevation = Column(Float)
    
    
    def __repr__(self):
        return f"id={self.id}, station= {self.station}, name = {self.name}, latitude ={self.latitude}, longitude = {self.longitude}, elevation = {self.elevation}"

In [9]:
Base.metadata.create_all(engine)

In [10]:
measurements= pd.read_csv("clean_Resources_hawaii_measurements.csv")

station= pd.read_csv("clean_Resources_hawaii_stations.csv")

In [11]:
measure_dict = measurements.to_dict(orient = 'records')
station_dict = station.to_dict(orient = 'records')

In [12]:
metadata = MetaData(bind=engine)
metadata.reflect()

In [13]:
table1 = sqlalchemy.Table('measurements', metadata, autoload=True)
table2 = sqlalchemy.Table('station', metadata, autoload=True)

In [14]:
conn.execute(table1.insert(), measure_dict)
conn.execute(table2.insert(), station_dict)

In [15]:

# conn.execute(table1.delete())
# conn.execute(table2.delete())

In [16]:
conn.execute(table1.insert(), measure_dict)
conn.execute(table2.insert(), station_dict)

In [17]:
conn.execute("select * from measurements limit 5").fetchall()

[(1, 'USC00519397', '2010-01-01', 0.08, 65.0),
 (2, 'USC00519397', '2010-01-02', 0.0, 63.0),
 (3, 'USC00519397', '2010-01-03', 0.0, 74.0),
 (4, 'USC00519397', '2010-01-04', 0.0, 76.0),
 (5, 'USC00519397', '2010-01-07', 0.06, 70.0)]

In [18]:
conn.execute("select * from station limit 5").fetchall()

[(1, 'USC00519397', 'WAIKIKI 717.2, HI US', 21.2716, -157.8168, 3.0),
 (2, 'USC00513117', 'KANEOHE 838.1, HI US', 21.4234, -157.8015, 14.6),
 (3, 'USC00514830', 'KUALOA RANCH HEADQUARTERS 886.9, HI US', 21.5213, -157.8374, 7.0),
 (4, 'USC00517948', 'PEARL CITY, HI US', 21.3934, -157.9751, 11.9),
 (5, 'USC00518838', 'UPPER WAHIAWA 874.3, HI US', 21.4992, -158.0111, 306.6)]

In [19]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [20]:
engine = create_engine("sqlite:///hawaii.sqlite")

In [21]:
Base = automap_base()
Base.prepare(engine, reflect= True)
Base.classes.keys()

['Measurements', 'Station']